In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.signal import find_peaks
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA

# Load data tables
Refer = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/RiverReferenceTable.csv')
Discharge = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Historical/HistoricalDischarge.csv')
Precip = pd.read_csv('C:/Users/Scott/Desktop/Projects/River_Data/Historical/HistoricalPrecip.csv')

# Standardize Date and River Names to be joined
Discharge['Date'] = pd.to_datetime(Discharge.DateTime).dt.date
Discharge = Discharge.set_index('Name').join(Refer.set_index('USGS Name')).reset_index()[[
    'Date','Value','VariableDescription','Name','RiverName','FishType']]

Precip = Precip.set_index('Name').join(Refer.set_index('Name')).reset_index()[[
    'Date','Precip','Name','RiverName','FishType']]

# Create Avg Daily Discharge
Avg_Discharge = Discharge.groupby(['Date','Name','RiverName','FishType']).mean().reset_index()

# Remove incorrect data
Avg_Discharge = Avg_Discharge.loc[Avg_Discharge.Value>0]

Avg_Discharge.head()

,Date,Name,RiverName,FishType,Value
0,2020-10-19,East Fork Carson,East Fork,Trout,31.896875
1,2020-10-19,East Walker,East Walker,Trout,51.300000
2,2020-10-19,Eel River Fort Seward,Eel,Steelhead,33.240625
3,2020-10-19,Eel River Scotia,Eel,Steelhead,65.256250
4,2020-10-19,Hat Creek,Hat Creek,Trout,122.437500


#### Eel River Daily Avg Discharge
Review the data quality and ensure the information looks correct

In [3]:
# Reduce data to Eel River only
eel = Avg_Discharge.loc[Avg_Discharge['Name']=='Eel River Scotia']
eel.reset_index(inplace=True)
eel = eel.drop(columns='index')

# Create column for slope of discharge
eel['CFS_Slope'] = eel.Value.diff()

# Retrieve values of Slope
SlopeValues = eel.CFS_Slope.values

# Define the threshold for what is a good day for fishing
#       - slope between day CFS values must be at least 1 STD from the mean 
border = eel.CFS_Slope.mean()+(eel.CFS_Slope.std()/2)

# Define troughs that are below the threshold
peaks, _ = find_peaks(-SlopeValues, height=border)

# Logic to build the best day flag
#       - if the value is a trough point or has a negative low and is above 1 STD from the mean then also consider a good day 
bestdays = []
for i in range(len(eel.Value)):
    if i in peaks:
        x = 1
    elif (eel.CFS_Slope[i]<0) & (eel.Value[i]>=border):
        x = 1
    else: x = 0
    bestdays.append(x)
eel['BestDays'] = bestdays

eel.head()

,Date,Name,RiverName,FishType,Value,CFS_Slope,BestDays
0,2020-10-19,Eel River Scotia,Eel,Steelhead,65.256250,NaN,0
1,2020-10-20,Eel River Scotia,Eel,Steelhead,65.536458,0.280208,0
2,2020-10-21,Eel River Scotia,Eel,Steelhead,64.127083,-1.409375,0
3,2020-10-22,Eel River Scotia,Eel,Steelhead,64.877083,0.750000,0
4,2020-10-23,Eel River Scotia,Eel,Steelhead,73.515625,8.638542,0


In [4]:
dates = eel.loc[eel.BestDays==1].Date.values
BestDayValues = eel.Value.loc[eel.BestDays==1].values

fig = px.line(eel, x="Date", y="Value", title='Eel River (Scotia) - Daily CFS')
fig.add_scatter(x=dates,y=BestDayValues,mode='markers',name='Good fishing days')
fig.show()

In [5]:
# Replicate for all rivers
Rivers = []
for river in Avg_Discharge.Name.unique():
    df = Avg_Discharge.loc[Avg_Discharge.Name == river]
    df.reset_index(inplace=True)
    df = df.drop(columns='index')

    # Create column for slope of discharge
    df['CFS_Slope'] = df.Value.diff()

    # Retrieve values of Slope
    SlopeValues = df.CFS_Slope.values

    # Define the threshold for what is a good day for fishing
    #       - slope between day CFS values must be at least 1 STD from the mean 
    border = df.CFS_Slope.mean()+(df.CFS_Slope.std()/2)

    # Define troughs that are below the threshold
    peaks, _ = find_peaks(-SlopeValues, height=border)

    # Logic to build the best day flag
    #       - if the value is a trough point or has a negative low and is above 1 STD from the mean then also consider a good day 
    bestdays = []
    for i in range(len(df.Value)):
        if i in peaks:
            x = 1
        elif (df.CFS_Slope[i]<0) & (df.Value[i]>=border):
            x = 1
        else: x = 0
        bestdays.append(x)
    df['BestDays'] = bestdays
    Rivers.append(df)
Rivers_Flow = pd.concat(Rivers)

In [6]:
Precip['Date'] = pd.to_datetime(Precip.Date).dt.date
Precip['Date_Name'] = Precip.Date.astype('str')+'_'+Precip.Name
Precip = Precip[['Date_Name','Date','Name','Precip']].groupby(['Date','Name','Date_Name']).mean().reset_index()

Rivers_Flow['Date_Name'] = Rivers_Flow.Date.astype('str')+'_'+Rivers_Flow.Name
Rivers_Flow['CFS'] = Rivers_Flow['Value']
Rivers_Flow.drop(columns=['Value'],inplace=True)

Rivers_Flow_Rain = Rivers_Flow.set_index('Date_Name').join(Precip[['Date_Name','Precip']].set_index('Date_Name'))
Rivers_Flow_Rain['Precip'] = Rivers_Flow_Rain.Precip.fillna(0)
RiverList = Rivers_Flow_Rain.Name.unique()
full = Rivers_Flow_Rain.loc[Rivers_Flow_Rain.Precip.isnull()==False]

# Lag Precip data by correlation strength to river flow
RiverData = pd.DataFrame()
for n in RiverList:
    s = 0
    CorrList = list()
    while s < 10:
        d = full[['CFS','Precip']].loc[full.Name==n]
        d['Precip'] = d['Precip'].shift(s)
        r = d.dropna()
        corr = r.Precip.corr(r.CFS)
        points = len(r.CFS)
        CorrList.append([s,corr,points])
        s+=1
    Corrs = pd.DataFrame(CorrList,columns=['PrecipLag','Corr','NumPoints'])
    BestShift = Corrs.sort_values(by='Corr',ascending=False).head(1)
    BestShift['Name'] = n
    RiverData = RiverData.append(BestShift)

In [7]:
full.head()

,Date,Name,RiverName,FishType,CFS_Slope,BestDays,CFS,Precip
Date_Name,,,,,,,,
2020-10-19_East Fork Carson,2020-10-19,East Fork Carson,East Fork,Trout,NaN,0,31.896875,0.000000
2020-10-20_East Fork Carson,2020-10-20,East Fork Carson,East Fork,Trout,0.297222,0,32.194097,0.000000
2020-10-21_East Fork Carson,2020-10-21,East Fork Carson,East Fork,Trout,0.683681,0,32.877778,0.000000
2020-10-22_East Fork Carson,2020-10-22,East Fork Carson,East Fork,Trout,0.357516,0,33.235294,0.714286
2020-10-23_East Fork Carson,2020-10-23,East Fork Carson,East Fork,Trout,1.090053,0,34.325347,0.000000


In [8]:
RiverData.loc[(RiverData.NumPoints>=30)&(RiverData.Corr>=.5)].sort_values('Corr',ascending=False )

,PrecipLag,Corr,NumPoints,Name


In [9]:
eel = full.loc[full['Name']=='Eel River Scotia']
eel.reset_index(inplace=True)
eel['Precip_Shift'] = eel.Precip.shift(1)
dates = eel.loc[eel.BestDays==1].Date.values
BestDayValues = eel.CFS.loc[eel.BestDays==1].values
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=eel.Date, y=eel.CFS, name="CFS"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=eel.Date, y=eel.Precip, name="Precip"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Precip & Discharge Timeline"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Discharge (cfs)", secondary_y=False)
fig.update_yaxes(title_text="Precip (mm)", secondary_y=True)
fig.add_scatter(x=dates,y=BestDayValues,mode='markers',name='Good fishing days')
fig.show()

<ipython-input-9-cb588a8cf424>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [10]:
# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=eel.Date, y=eel.CFS, name="CFS"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=eel.Date, y=eel.Precip_Shift, name="Precip"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Shift Precip to align with Discharge"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Discharge (cfs)", secondary_y=False)
fig.update_yaxes(title_text="Precip (mm)", secondary_y=True)
fig.add_scatter(x=dates,y=BestDayValues,mode='markers',name='Good fishing days')
fig.show()

In [15]:


rolling_mean = eel.CFS.rolling(window = 12).mean()
rolling_std = eel.CFS.rolling(window = 12).std()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": False}]])

# Add traces
fig.add_trace(
    go.Scatter(x=eel.Date, y=eel.CFS, name="CFS"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=eel.Date, y=rolling_mean, name="rolling_mean"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=eel.Date, y=rolling_std, name="rolling_std"),
    secondary_y=False,
)
# Add figure title
fig.update_layout(
    title_text="Rolling Mean & Rolling Standard Deviation'"
)

# Set x-axis title
fig.update_xaxes(title_text="Date")

# Set y-axes titles
fig.update_yaxes(title_text="Discharge (cfs)", secondary_y=False)
fig.show()

In [53]:
eel.CFS_Stationary = eel.CFS_Slope

fig = px.line(x=eel.Date,y=eel.CFS_Stationary)
fig.show()